<a href="https://colab.research.google.com/github/liviabelirocha/webscraping-studies/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install python-slugify
!apt-get update
!apt install chromium-chromedriver

In [2]:
from selenium import webdriver
from slugify import slugify
import pandas as pd

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

In [4]:
wd.get("https://www.thespruce.com/plants-a-to-z-5116344")
cards = wd.find_elements_by_class_name('alphabetical-card')

In [7]:
links = []
for card in cards:
  links.append(card.get_attribute('href'))
del links[:8]

In [27]:
def split_list(array, elem, type = 1):
  for idx, el in enumerate(array):
    if el == elem:
      return array[:idx] if type == 1 else array[idx+1:]

def convert_dict(array):  
    init = iter(array)
    res_dct = dict(zip(init, init))  
    return res_dct

def remove_words(array):
  for word in array:
    if word.replace(' ', '').isupper():
      array.remove(word)
  return array

def separate_arrays(array):
  titles = []
  contents = []
  aux_contents = []

  count = 0

  for elem in array:
    if len(elem.split(' ')) <= 10:
      titles.append(elem)
      if count != 0:
        contents.append(aux_contents)
        aux_contents = []
      count += 1
    else:
      aux_contents.append(elem)

  return titles, contents

In [47]:
def inspect_plant(link):
  wd.get(link)
  link = wd.find_element_by_xpath('/html/body/main/article/div[1]/div/div[2]/div[2]')

  page_list = link.text.split('\n')
  remove_words(page_list)

  table = link.find_element_by_tag_name('table')

  # Get description
  description = split_list(page_list, table.text.split('\n')[0])
  description = ''.join(description)

  content = split_list(page_list, table.text.split('\n')[-1], 2)

  titles, contents = separate_arrays(content)

  for (idx, title) in enumerate(titles):
    titles[idx] = slugify(title)

  for (idx, content) in enumerate(contents):
    contents[idx] = ' '.join(content)

  if len(titles) > len(contents):
    titles.pop()

  zip_iterator = zip(titles, contents)
  content_dict = dict(zip_iterator)

  # Get table
  table = table.find_elements_by_tag_name('td')
  dict_table = []
  for (idx, column) in enumerate(table):
    text = column.text
    if idx % 2 == 0:
      text = slugify(text)
    dict_table.append(text)
  dict_table = convert_dict(dict_table)

  return {'description': description, **dict_table, **content_dict}

In [48]:
res = []
for link in links[:3]:
  res.append(inspect_plant(link))

df = pd.DataFrame(res)

In [49]:
df

,description,botanical-name,common-name,plant-type,mature-size,sun-exposure,soil-type,soil-ph,bloom-time,flower-color,hardiness-zones,native-area,how-to-grow-acoma-crape-myrtle,light,soil,water,temperature-and-humidity,fertilizer,propagating-acoma-crape-myrtle,follow-these-steps-to-propagate-with-cuttings,toxicity-of-acoma-crape-myrtle,pruning,common-pests,latin-name,common-names,toxicity,aeonium-care,aeonium-varieties,the-spruce-kara-riley,potting-and-repotting,propagating-aeoniums,aerangis-orchid-care,the-spruce-phoebe-cheong,aerangis-orchid-varieties,propagating-aerangis-orchids,common-pests-diseases
0,The Acoma crape myrtle is a hybrid variety of ...,Lagerstroemia x ‘Acoma’,Acoma crape myrtle,Tree,2 to 10 feet tall and 2 to 10 feet wide,Full sun,Well-draining,Slightly acidic to neutral,"Spring, summer",White,7 to 9,"Asia, Australia",The Acoma crape myrtle is a rewarding shrub or...,Full sun is necessary for crape myrtle to bloo...,The crape myrtle is adaptable to varying soil ...,"When first planted, the crape myrtle will need...",Like other crape myrtle varieties that thrive ...,"For the best blooms, you may need to fertilize...",The most successful way of propagating Acoma c...,Use clean scissors or garden shears to remove ...,If you’re concerned about your pets or other a...,Keep your Acoma crape myrtle in good form with...,While crepe myrtle trees are often subject to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aeonium is a genus including about 35 succulen...,NaN,NaN,Perennial succulent,2–60 inches (depending on species and variety),Full sun to part shade,Sandy loam,5.6–6.0 (slightly acidic),Late winter or spring,"Pink (flowering is rare, occurring only in mat...",9–11(USDA); often grown as potted plants broug...,"Canary Islands, Africa",NaN,"As with most succulents, Aeonium plants grow b...",A sandy loam or regular potting mix amended wi...,"In the winter, water whenever the top inch of ...",These plants prefer a Mediterranean climate—no...,Feed during the growing season with a half-str...,NaN,NaN,NaN,NaN,NaN,Aeonium spp.,Aeonium,Non-toxic,"In warmer climates, Aeoniums can be grown as i...",Aeonium arboreum: This widely available plant ...,,"Needing so little soil, Aeoniums are great for...","Like many succulents, Aeoniums are very easy t...",NaN,NaN,NaN,NaN,NaN
2,The Aerangis genus comprises many tropical orc...,Aerangis spp.,Aerangis orchids,Epiphytic orchids,6–24 inches (varies by species),"Part sun, bright filtered light",Orchid potting mix,5.5 to 6.0 (acidic),Spring to fall (varies by species),"White, yellow",10–11 (USDA),"Tropical Africa, Madagascar",NaN,These tropical orchids need lots of bright ind...,"A well-draining epiphyte mix is best, such as ...",Aerangis orchids need a very moist environment...,These plants require warm tropical temperature...,"Feed regularly with a balanced, diluted fertil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-toxic,NaN,NaN,NaN,"Many people mount Aerangis plants on a sheer, ...",NaN,"As is true of most tropical orchids, successfu...",,"One relatively popular Aerangis is A. citrata,...",These epiphytes can be propagated by division....,"Orchids can be temperamental plants, susceptib..."
